In [1]:
# coding:utf-8  
'''
@author: Jason.F
@data: 2019.08.16
@function: BNMF (Bayesian Neural Matrix Factorization)
           Datatset: MovieLens-1m:https://grouplens.org/datasets/movielens/  
           Evaluation:RMSE
@reference:minibatch training, https://twiecki.io/blog/2016/07/05/bayesian-deep-learning/ 
'''
import sys
import time
import heapq
import math
import gc
import numpy as np
import pandas as pd

import pymc3 as pm
import theano
import tensorflow as tf
import theano.tensor as tt

class DataSet:
    def __init__(self):
        self.trainset, self.testset, self.maxu, self.maxi, self.maxr = self._getDataset_as_list()
        
    def _getDataset_as_list(self):
        #trainset
        filePath = "/data/fjsdata/BMF/ml-1m.train.rating" 
        data = pd.read_csv(filePath, sep='\t', header=None, names=['user', 'item', 'rating'], \
                                 usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2: np.float})
        maxu, maxi, maxr = data['user'].max()+1, data['item'].max()+1, data['rating'].max()
        print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
                  (data.shape[0], maxu, maxi, data.shape[0]/(maxu*maxi)))
        trainset = data.values.tolist()
        #testset
        filePath = "/data/fjsdata/BMF/ml-1m.test.rating" 
        data = pd.read_csv(filePath, sep='\t', header=None, names=['user', 'item', 'rating'], \
                                 usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2: np.float})
        testset = data.values.tolist()
        return trainset, testset, maxu, maxi, maxr 
        
    def list_to_matrix(self, dataset, maxu, maxi):              
        dataMat = np.zeros([maxu, maxi], dtype=np.float32)
        for u,i,r in dataset:
            dataMat[int(u)][int(i)] = float(r)
        return np.array(dataMat)
    
    def list_to_dict(self, dataset):
        dataDict = {}
        for u,i,r in dataset:
            dataDict[int(u), int(i)] = float(r)
        return dataDict
    
    def getInstances(self, dataset):
        user = []
        item = []
        rate = []
        for u, i, r in dataset:
            user.append(int(u))
            item.append(int(i))
            rate.append(float(r))
        return np.array(user), np.array(item), np.array(rate)
    
class BNMF:
    def __init__(self, dataset, K):
        self.shape = [dataset.maxu, dataset.maxi]
        self.maxr = dataset.maxr
        self.R = dataset.list_to_matrix(dataset.trainset, dataset.maxu, dataset.maxi)

        #K is number of latent factors
        self.userLayer = [1024, K]
        self.itemLayer = [1024, K]

    def train_BNMF(self, verbose=10):       
        print('start training the BNMF model')
        tstart = time.time()
        
        with pm.Model() as self.bnmf:#difine
            #user layer
            user_W1 = pm.Normal('user_W1', 0, sd=1, shape=[self.shape[1], self.userLayer[0]] )
            user_O1 = tt.nnet.relu(tt.dot(self.R, user_W1))
            user_W2 = pm.Normal('user_W2', 0, sd=1, shape=[self.userLayer[0],self.userLayer[1]] )
            user_O2 = tt.nnet.relu(tt.dot(user_O1, user_W2))
            #item layer
            item_W1 = pm.Normal('item_W1', 0, sd=1, shape=[self.shape[0], self.itemLayer[0]] )
            item_O1 = tt.nnet.relu(tt.dot(self.R.T, item_W1))
            item_W2 = pm.Normal('item_W2', 0, sd=1, shape=[self.itemLayer[0],self.itemLayer[1]] )
            item_O2 = tt.nnet.relu(tt.dot(item_O1, item_W2))
            # latent factor matrix
            R = pm.Deterministic('R', tt.dot(user_O2,item_O2.T))
            Y = pm.Normal('Y',mu=R, sd=self.maxr/2, observed=self.R)
            
        with self.bnmf:#inference
            approx = pm.fit(n=10000, method=pm.ADVI())
            trace = approx.sample(draws=2000)
            
        elapsed = time.time() - tstart    
        print('Completed training the BNMF model in %d seconds' % int(elapsed))
        
        return trace
           
    def eval_BNMF(self, trace, testset):
        
        with self.bnmf:#evaluation
            ppc = pm.sample_posterior_predictive(trace, progressbar=True) 
            nR =  np.mean(ppc['Y'],0) #expectation of parameters
            
        squaredError = []
        for u,i,r in testset:    
            error=r - nR[int(u),int(i)]
            squaredError.append(error * error)
        rmse =math.sqrt(sum(squaredError) / len(squaredError))
        return rmse
    
if __name__ == "__main__":
    ds = DataSet()#loading dataset
    for K in [8,16,32,64]:
        model = BNMF(ds, K)  
        trace = model.train_BNMF()
        rmse = model.eval_BNMF(trace=trace, testset=ds.testset)
        print("RMSE@{}:{}".format(K, rmse))

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Dataset Statistics: Interaction = 994169, User = 6040, Item = 3706, Sparsity = 0.0444
start training the BNMF model


Average Loss = 3.9334e+10: 100%|██████████| 10000/10000 [15:04:37<00:00,  4.46s/it] 
Finished [100%]: Average Loss = 3.8741e+10
I0816 23:55:21.916945 139764077238016 inference.py:248] Finished [100%]: Average Loss = 3.8741e+10


ValueError: rng_mrg cpu-implementation does not support more than (2**31 -1) samples
Apply node that caused the error: mrg_uniform{TensorType(float64, vector),inplace}(<TensorType(int32, matrix)>, MakeVector{dtype='int64'}.0)
Toposort index: 11
Inputs types: [TensorType(int32, matrix), TensorType(int64, vector)]
Inputs shapes: [(15360, 6), (1,)]
Inputs strides: [(24, 4), (8,)]
Inputs values: ['not shown', array([19992576000])]
Outputs clients: [['output'], [Shape_i{0}(mrg_uniform{TensorType(float64, vector),inplace}.1), Subtensor{int64::}(mrg_uniform{TensorType(float64, vector),inplace}.1, ScalarFromTensor.0), Subtensor{:int64:}(mrg_uniform{TensorType(float64, vector),inplace}.1, ScalarFromTensor.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the T

In [1]:
# coding:utf-8  
'''
@author: Jason.F
@data: 2019.08.02
@function: BNMF (Bayesian Neural Matrix Factorization)
           Datatset: MovieLens-1m:https://grouplens.org/datasets/movielens/  
           Evaluation: HR@10 NDCG@10
'''

import sys
import time
import heapq
import math
import gc
import numpy as np
import pandas as pd

import pymc3 as pm
import theano
import tensorflow as tf
import theano.tensor as tt

class DataSet:
    def __init__(self):
        self.trainset, self.testset, self.maxu, self.maxi = self._getDataset_as_list()
        
    def _getDataset_as_list(self):
        #trainset
        filePath = "/data/fjsdata/BNMF/ml-1m.train.rating" 
        data = pd.read_csv(filePath, sep='\t', header=None, names=['user', 'item', 'rating'], \
                                 usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2: np.float})
        data['rating']=data['rating'].apply(lambda x: 1.0 if float(x)>0.0 else 0.0)
        maxu, maxi = data['user'].max()+1, data['item'].max()+1
        print('Dataset Statistics: Interaction = %d, User = %d, Item = %d, Sparsity = %.4f' % \
                  (data.shape[0], maxu, maxi, data.shape[0]/(maxu*maxi)))
        trainset = data.values.tolist()
        #testset
        testset = []
        filePath = "/data/fjsdata/BNMF/ml-1m.test.negative" 
        with open(filePath, 'r') as fd:
            line = fd.readline()
            while line != None and line != '':
                arr = line.split('\t')
                u = eval(arr[0])[0]
                testset.append([u, eval(arr[0])[1], 1.0])#first is one postive item
                for i in arr[1:]:
                    testset.append([u, int(i), 0.0]) #99 negative items
                line = fd.readline()
        return trainset, testset, maxu, maxi
        
    def list_to_matrix(self, dataset, maxu, maxi):              
        dataMat = np.zeros([maxu, maxi], dtype=np.float32)
        for u,i,r in dataset:
            dataMat[int(u)][int(i)] = float(r)
        return np.array(dataMat)
    
    def list_to_dict(self, dataset):
        dataDict = {}
        for u,i,r in dataset:
            dataDict[int(u), int(i)] = float(r)
        return dataDict
    
    def getInstances(self, dataset, maxu=None, maxi=None, num_ng=None, isTest=False):
        user = []
        item = []
        rate = []
        if isTest==True: #test
            for u, i, r in dataset:
                user.append(int(u))
                item.append(int(i))
                rate.append(float(r))
        else:#train
            for u, i, r in dataset:
                user.append(int(u))
                item.append(int(i))
                rate.append(float(r))
            #negative samples
            dataDict = self.list_to_dict(dataset)
            for j in range(len(dataset)*num_ng):
                u = np.random.randint(maxu)
                i = np.random.randint(maxi)
                while (u, i) in dataDict:
                    u = np.random.randint(maxu)
                    i = np.random.randint(maxi)
                user.append(int(u))
                item.append(int(i))
                rate.append(float(0.0)) 
        return np.array(user), np.array(item), np.array(rate)
    
class BNMF:
    def __init__(self, dataset, K, num_ng):
        self.shape = [dataset.maxu, dataset.maxi]
        #get the trainset and testset
        self.train_u, self.train_i, self.train_r = dataset.getInstances(dataset = dataset.trainset, \
                                                                        maxu=dataset.maxu, \
                                                                        maxi=dataset.maxi, \
                                                                        num_ng=num_ng, isTest=False)
        assert(len(self.train_u) == len(self.train_i) and len(self.train_i) == len(self.train_r))
        shuffled_idx = np.random.permutation(np.arange(len(self.train_u)))
        self.train_u = self.train_u[shuffled_idx]
        self.train_i = self.train_i[shuffled_idx]
        self.train_r = self.train_r[shuffled_idx]
        self.test_u, self.test_i, self.test_r = dataset.getInstances(dataset.testset, isTest=True)
        assert(len(self.test_u) == len(self.test_i) and len(self.test_i) == len(self.test_r))
        
        #initialize
        #K is number of latent factors
        self.userLayer = [512, K]
        self.itemLayer = [512, K]
        self.batchSize = 1024
        self.lr = 0.001
        self.reg = 0.01
        tf.reset_default_graph()
        self._build_MLP(dataset)
        self._init_sess()
        
    def _init_sess(self):
        #define seesion
        self.config = tf.ConfigProto()
        self.config.gpu_options.allow_growth = True
        self.config.allow_soft_placement = True
        self.sess = tf.Session(config=self.config)
        self.sess.run(tf.global_variables_initializer())
        
    def _build_MLP(self, dataset):
        print('start building the Multi-layer non-linear projection')
        # add placeholder
        self.user = tf.placeholder(tf.int32)
        self.item = tf.placeholder(tf.int32)
        self.rate = tf.placeholder(tf.float32)
        user_item_embedding = tf.convert_to_tensor(dataset.list_to_matrix(dataset.trainset, dataset.maxu, dataset.maxi))
        item_user_embedding = tf.transpose(user_item_embedding)
        user_input = tf.nn.embedding_lookup(user_item_embedding, self.user)
        item_input = tf.nn.embedding_lookup(item_user_embedding, self.item)
        
        def init_variable(shape, name):
            return tf.Variable(tf.truncated_normal(shape=shape, dtype=tf.float32, stddev=0.01), name=name)

        with tf.name_scope("User_Layer"):
            user_W1 = init_variable([self.shape[1], self.userLayer[0]], "user_W1")
            user_out = tf.matmul(user_input, user_W1)
            for i in range(0, len(self.userLayer)-1):
                W = init_variable([self.userLayer[i], self.userLayer[i+1]], "user_W"+str(i+2))
                b = init_variable([self.userLayer[i+1]], "user_b"+str(i+2))
                user_out = tf.nn.relu(tf.add(tf.matmul(user_out, W), b))

        with tf.name_scope("Item_Layer"):
            item_W1 = init_variable([self.shape[0], self.itemLayer[0]], "item_W1")
            item_out = tf.matmul(item_input, item_W1)
            for i in range(0, len(self.itemLayer)-1):
                W = init_variable([self.itemLayer[i], self.itemLayer[i+1]], "item_W"+str(i+2))
                b = init_variable([self.itemLayer[i+1]], "item_b"+str(i+2))
                item_out = tf.nn.relu(tf.add(tf.matmul(item_out, W), b))
           
        self.r_ui = tf.reduce_sum(tf.multiply(user_out, item_out), axis=1, keepdims=False)
        loss = tf.reduce_sum(tf.losses.mean_squared_error(labels = self.rate, predictions=self.r_ui))
        regLoss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])#reguralizer
        self.loss = loss + self.reg * regLoss
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_step = optimizer.minimize(self.loss)
        print('done building the the Multi-layer non-linear projection')
        
    def _build_BPF(self):
        print('start building the Bayesian probabilistic model')  
        self.x_u = theano.shared(self.train_u)
        self.x_i = theano.shared(self.train_i)
        self.y_r = theano.shared(self.train_r)
        self.y_r_ui = theano.shared(np.array(self.nn_r_ui))
        assert(len(self.y_r.get_value())==len(self.y_r_ui.get_value()))
        with pm.Model() as self.bncf:#define the prior and likelihood
            b_u = pm.Normal('b_u', 0, sd=1/2, shape=self.shape[0])
            b_i = pm.Normal('b_i', 0, sd=1/2, shape=self.shape[1])
            u = pm.Normal('u', 0, sd=1/2)
            tY = pm.Deterministic('tY', tt.add(tt.add(tt.add(b_u[self.x_u],b_i[self.x_i]),self.y_r_ui),u))
            #tY = pm.Deterministic('tY', ((b_u[self.x_u]+b_i[self.x_i])+self.y_r_ui)+u)#b_u+b_i+u+nn_r_ui
            nY = pm.Deterministic('nY', pm.math.sigmoid(tY))
            # likelihood of observed data
            Y = pm.Bernoulli('Y', nY, observed=self.y_r)#Categorical 
        with self.bncf:#inference
            approx = pm.fit(n=10000, method=pm.ADVI())
            self.trace = approx.sample(draws=3000)
        with self.bncf: #posterior prediction
            ppc = pm.sample_posterior_predictive(self.trace, progressbar=True)
            self.by_r_ui = ppc['Y'].mean(axis=0)
        print('done building the Bayesian probabilistic model')
        
    def train_BNMF(self, verbose=10):       
        print('start training the BNCF model')
        tstart = time.time()
        
        num_batches = len(self.train_u) // self.batchSize + 1
        #1.traing r_ui in neural network 
        self.nn_r_ui=[]
        for i in range(num_batches):
            min_idx = i * self.batchSize
            max_idx = np.min([len(self.train_u), (i+1)*self.batchSize])
            train_u_batch = self.train_u[min_idx: max_idx]
            train_i_batch = self.train_i[min_idx: max_idx]
            #train_r_batch = self.train_r[min_idx: max_idx]
            pre_r_ui_batch = self.sess.run(self.r_ui, feed_dict={self.user: train_u_batch, \
                                                                 self.item: train_i_batch})
            self.nn_r_ui.extend(pre_r_ui_batch)
            if verbose and i % verbose == 0:
                sys.stdout.write('\r{} / {} : shape = {} '.format(i, num_batches, len(self.nn_r_ui)))
                sys.stdout.flush()
        #2.training bias in Bayesian inference
        self._build_BPF()
        #3.training self.loss in neural network
        losses = []
        for i in range(num_batches):
            min_idx = i * self.batchSize
            max_idx = np.min([len(self.train_u), (i+1)*self.batchSize])
            train_u_batch = self.train_u[min_idx: max_idx]
            train_i_batch = self.train_i[min_idx: max_idx]
            train_r_batch = self.by_r_ui[min_idx: max_idx]
            _, tmp_loss = self.sess.run([self.train_step, self.loss], feed_dict={self.user: train_u_batch, \
                                                                                 self.item: train_i_batch, \
                                                                                 self.rate: train_r_batch})
            losses.append(tmp_loss)
            if verbose and i % verbose == 0:
                sys.stdout.write('\r{} / {} : loss = {}'.format(i, num_batches, np.mean(losses[-verbose:])))
                sys.stdout.flush()
        loss = np.mean(losses)  
        elapsed = time.time() - tstart    
        print('Completed training the BNCF model in %d seconds' % int(elapsed))
        return loss
           
    def eval_BNMF(self, verbose=10):
        def getHitRatio(ranklist, targetItem):
            for item in ranklist:
                if item == targetItem:
                    return 1
            return 0
    
        def getNDCG(ranklist, targetItem):
            for i in range(len(ranklist)):
                item = ranklist[i]
                if item == targetItem:
                    return math.log(2) / math.log(i+2)
            return 0
    
        #1.get r_ui in neural network
        num_batches = len(self.test_u) // self.batchSize + 1
        #1.traing r_ui in neural network 
        self.nn_r_ui=[]
        for i in range(num_batches):
            min_idx = i * self.batchSize
            max_idx = np.min([len(self.test_u), (i+1)*self.batchSize])
            test_u_batch = self.test_u[min_idx: max_idx]
            test_i_batch = self.test_i[min_idx: max_idx]
            pre_r_ui_batch = self.sess.run(self.r_ui, feed_dict={self.user: test_u_batch, \
                                                                 self.item: test_i_batch})
            self.nn_r_ui.extend(pre_r_ui_batch)
            if verbose and i % verbose == 0:
                sys.stdout.write('\r{} / {} : shape = {} '.format(i, num_batches, len(self.nn_r_ui)))
                sys.stdout.flush()
                
        #2. get biais in Bayesian inference
        self.x_u.set_value(self.test_u)
        self.x_i.set_value(self.test_i)
        self.y_r.set_value(self.test_r)
        self.y_r_ui.set_value(self.nn_r_ui)
        with self.bncf:#evaluation
            ppc = pm.sample_posterior_predictive(self.trace, progressbar=True) 
            pre_r = ppc['Y'].mean(axis=0)
        assert(pre_r.shape[0]==self.test_i.shape[0])
        #every user have one positive item and 99 negative items
        hrs = []
        ndcgs = []
        num_batches = len(self.test_i) // 100
        for i in range(num_batches):
            test_batch = self.test_i[i*100: (i+1)*100]
            pre_r_batch = pre_r[i*100: (i+1)*100]
            targetItem = int(test_batch[0])#positive item
            map_item_score = {}
            for j in range(100):
                map_item_score[int(test_batch[j])] = pre_r_batch[j]
            ranklist = heapq.nlargest(10, map_item_score, key=map_item_score.get)#default Topn=10
            hrs.append(getHitRatio(ranklist, targetItem))
            ndcgs.append(getNDCG(ranklist, targetItem))
        hr, ndcg = np.array(hrs).mean(), np.array(ndcgs).mean()
        return hr, ndcg
    
if __name__ == "__main__":
    ds = DataSet()#loading dataset
    for K in [8, 16, 32, 64]:
        model = BNMF(ds, K, num_ng=2)
        best_hr = 0.0
        best_ndcg = 0.0
        for epoch in range(2):
            loss = model.train_BNMF()
            print("\nMean loss in this epoch is: {}".format(loss))
            hr, ndcg = model.eval_BNMF()
            if hr>best_hr: best_hr=hr
            if ndcg>best_ndcg: best_ndcg=ndcg
        print("HR@10:{}, NDCG@10:{}, K:{}".format(best_hr, best_ndcg, K))

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Dataset Statistics: Interaction = 994169, User = 6040, Item = 3706, Sparsity = 0.0444
start building the Multi-layer non-linear projection


W0803 14:39:07.944793 140164905830144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


done building the the Multi-layer non-linear projection
start training the BNCF model
2910 / 2913 : shape = 2980864 start building the Bayesian probabilistic model


Average Loss = 1.1495e+06: 100%|██████████| 10000/10000 [1:21:06<00:00,  1.99it/s]
Finished [100%]: Average Loss = 1.1495e+06
I0803 16:01:42.123292 140164905830144 inference.py:248] Finished [100%]: Average Loss = 1.1495e+06
100%|██████████| 3000/3000 [25:17<00:00,  2.18it/s] 


...

start training the BNCF model
2910 / 2913 : shape = 2980864 start building the Bayesian probabilistic model


Average Loss = 1.138e+06: 100%|██████████| 10000/10000 [1:26:13<00:00,  1.94it/s]
Finished [100%]: Average Loss = 1.1379e+06
I0804 08:11:11.584494 140164905830144 inference.py:248] Finished [100%]: Average Loss = 1.1379e+06
100%|██████████| 3000/3000 [30:24<00:00,  1.90it/s]


done building the Bayesian probabilistic model
2910 / 2913 : loss = 4.186251163482666881Completed training the BNCF model in 8526 seconds

Mean loss in this epoch is: 5.048369407653809
580 / 590 : shape = 594944 

100%|██████████| 3000/3000 [05:26<00:00, 10.07it/s]


HR@10:0.46341059602649004, NDCG@10:0.25838422916669485, K:64
